# Exercício 1
## Demanda do Especialista do Ministério da Agricultura
- Avaliação do Impacto de Fatores Climáticos na Produção Agrícola


---

Importando bibliotevas necessárias

In [1]:
import pandas as pd
from hashlib import sha1
import os

## Dados sobre Grãos

### Lendo dados sobre área plantada

In [167]:
df_planted_area = pd.read_csv(
    filepath_or_buffer='data/raw/grains/planted_area.csv',
    sep=',',
    index_col=0,
)

Transformando tabela de tipo largo para longo

In [168]:
df_planted_area = pd.melt(
    frame=df_planted_area,
    id_vars=['Grain', 'Year'],
    var_name='city',
    value_name='planted_area'
)

Visualizando dados

In [169]:
df_planted_area.tail()

,Grain,Year,city,planted_area
2558975,Pea,2017,Brasília (DF),32.0
2558976,Barley,2018,Brasília (DF),100.0
2558977,Pea,2018,Brasília (DF),40.0
2558978,Barley,2019,Brasília (DF),100.0
2558979,Pea,2019,Brasília (DF),40.0


### Lendo dados sobre área colhida

In [170]:
df_harvested_area = pd.read_csv(
    filepath_or_buffer='data/raw/grains/harvested_area.csv',
    sep=',',
    index_col=0,
)

Transformando tabela de tipo largo para longo

In [171]:
df_harvested_area = pd.melt(
    frame=df_harvested_area,
    id_vars=['Grain', 'Year'],
    var_name='city',
    value_name='harvested_area'
)

Visualizando dados

In [172]:
df_harvested_area.tail()

,Grain,Year,city,harvested_area
2558975,Pea,2017,Brasília (DF),32.0
2558976,Barley,2018,Brasília (DF),100.0
2558977,Pea,2018,Brasília (DF),40.0
2558978,Barley,2019,Brasília (DF),70.0
2558979,Pea,2019,Brasília (DF),40.0


### Lendo dados sobre produtividade

In [173]:
df_yield = pd.read_csv(
    filepath_or_buffer='data/raw/grains/yield.csv',
    sep=',',
    index_col=0,
)

Transformando tabela de tipo largo para longo

In [174]:
df_yield = pd.melt(
    frame=df_yield,
    id_vars=['Grain', 'Year'],
    var_name='city',
    value_name='yield'
)

Visualizando dados

In [175]:
df_yield.tail()

,Grain,Year,city,yield
2558975,Pea,2017,Brasília (DF),7.0
2558976,Barley,2018,Brasília (DF),4.5
2558977,Pea,2018,Brasília (DF),7.0
2558978,Barley,2019,Brasília (DF),4.5
2558979,Pea,2019,Brasília (DF),7.0


### Lendo dados sobre produção

In [198]:
df_production = pd.read_csv(
    filepath_or_buffer='data/raw/grains/production.csv',
    sep=',',
    index_col=0
)

Transformando tabela de tipo largo para longo

In [199]:
df_production = pd.melt(
    frame=df_production,
    id_vars=['Grain', 'Year'],
    var_name='city',
    value_name='production'
)

Visualizando dados

In [210]:
df_production.tail()

,Grain,Year,city,production
2558975,Pea,2017,Brasília (DF),7.0
2558976,Barley,2018,Brasília (DF),4.5
2558977,Pea,2018,Brasília (DF),7.0
2558978,Barley,2019,Brasília (DF),4.5
2558979,Pea,2019,Brasília (DF),7.0


### Realizando cruzamento dos dados

Comparando se dados sobre produção e produtividade são diferentes

In [179]:
df_production['production'].equals(df_yield['yield'])

True

Neste dataset não são, logo podemos ficar com apenas um destes

Realizando mesca entre dados de plantio, colheita e produção

In [190]:
df_plantation = pd.merge(
    left=df_harvested_area,
    right=df_planted_area,
    how='inner',
    on=['Grain', 'Year', 'city']
)

df_plantation = pd.merge(
    left=df_plantation,
    right=df_production,
    how='left',
    on=['Grain', 'Year', 'city']
)

Extraindo unidade federativa do nome da cidade

In [191]:
df_plantation['state'] = df_plantation['city'].str.extract(r'\((\w{2})\)')

Removendo informação de unidade federativa da coluna de cidade

In [192]:
df_plantation['city'] = df_plantation['city'].str.replace(r'\(\w{2}\)', '', regex=True)
df_plantation['city'] = df_plantation['city'].str.strip()

Criando coluna com as regiões do brazil

In [202]:
brazil_regions = {
    'North': ['RO', 'AC', 'AM', 'RR', 'PA', 'AP', 'TO'],
    'Northeast': ['MA', 'PI', 'CE', 'RN', 'PB', 'PE', 'AL', 'SE', 'BA'],
    'Midwest': ['MT', 'MS', 'GO', 'DF'],
    'Southeast': ['MG', 'ES', 'RJ', 'SP'],
    'South': ['PR', 'SC', 'RS']
}

state_to_region = {state: region for region, states in brazil_regions.items() for state in states}

df_plantation['region'] = df_plantation['state'].map(state_to_region)

Criando chave artificial

In [203]:
df_plantation['artificial_key'] = (
    df_plantation['state']
    + df_plantation['city']
    + df_plantation['Grain']
    + df_plantation['Year'].astype(str)
).map(lambda x: sha1(x.encode('utf-8')).hexdigest())

Alterando nome de todas as colunas para _lower case_

In [204]:
df_plantation.columns = df_plantation.columns.map(lambda column: column.lower())

Alterando tipos das colunas

In [206]:
df_plantation = df_plantation.astype({
    'artificial_key': str,
    'region': str,
    'state': str,
    'city': str,
    'grain': str,
    'year': pd.UInt16Dtype(),
    'planted_area': pd.UInt64Dtype(),
    'harvested_area': pd.UInt64Dtype(),
    'production': pd.Float64Dtype(),
})

Alterando ordem das colunas

In [211]:
df_plantation = df_plantation.loc[:, ['artificial_key', 'region', 'state', 'city', 'grain', 'year', 'planted_area', 'harvested_area', 'production']]

Configurando novo índice para tabela

In [213]:
df_plantation.set_index('artificial_key', inplace=True)

Visualizando dados

In [221]:
df_plantation.tail()

,region,state,city,grain,year,planted_area,harvested_area,production
artificial_key,,,,,,,,
f45ad7729c422c093bfbf0fb1f99f8c6ce6dbbf6,Midwest,DF,Brasília,Pea,2017,32,32,7.0
9a0f47a14c2e92a914b7b6e0d264ed576667edff,Midwest,DF,Brasília,Barley,2018,100,100,4.5
806f31b6182bbc90bb5e1d6d5196dc670359d217,Midwest,DF,Brasília,Pea,2018,40,40,7.0
baf47bd13e9014dd74b727e08ca13223252f4ce0,Midwest,DF,Brasília,Barley,2019,100,70,4.5
2ec357c46a3bb717daea073782baa6b42278d772,Midwest,DF,Brasília,Pea,2019,40,40,7.0


Salvando dados em formato .parquet particioando por ano

In [232]:
df_plantation.to_parquet(
    path='data/trusted/grains/plantation',
    engine='pyarrow',
    index=True,
    compression='snappy',
    partition_cols=['year']
)

## Dados metereológicos

Mapeando o caminho de todos os arquivos csv com dados metereológicos

In [8]:
folder_path = './data/raw/environmental'
csv_files = list(map(
    lambda file: f'{folder_path}/{file}',
    os.listdir(folder_path)
))

Criando função para ler metadados sobre a estação meteorológica

In [76]:
def extract_station_info(file_path: str, number_of_lines: int = 9):

    metadata = {}

    with open(file_path) as file:
        
        line_number = 1
        while line_number < 10:
            line = file.readline().strip()
            key, value = line.split(':')
            metadata[key.strip()] = value.strip()
            line_number += 1
    
    return metadata

Lendos todos os arquivos CSV

In [203]:
df_station = pd.DataFrame()
df_meteorology = pd.DataFrame()

for csv_file in csv_files:

    station_metadata = extract_station_info(csv_file)
    df_station_data = pd.DataFrame([station_metadata])
    df_station = pd.concat([df_station, df_station_data])

    df_meteorology_data = pd.read_csv(
        filepath_or_buffer=csv_file,
        sep=';',
        skiprows=10, # ignorar linhas com metadados
        index_col=False,
        usecols=range(7), # coletar apenas as 7 primeiras colunas do CSV,
        decimal=',',
        parse_dates=[0], # convertendo primeira coluna para tipo date
        dtype={ 1: pd.UInt16Dtype() } # convertendo segunda coluna para int
    )

    df_meteorology_data['station_code'] = station_metadata['Codigo Estacao']
    df_meteorology = pd.concat([df_meteorology, df_meteorology_data])

In [204]:
df_station[['city', 'sub_region']] = df_station['Nome'].str.split(
    pat='-',
    n=1,
    expand=True
)

Corrigindo algumas exceções

In [ ]:
paranoa_index = df_station['Nome'] == 'PARANOA (COOPA-DF)'
df_station['city'][paranoa_index] = 'PARANOA'
df_station['sub_region'][paranoa_index] = None

guarda_mor_index = df_station['Nome'] == 'GUARDA-MOR'
df_station['city'][guarda_mor_index] = 'GUARDA-MOR'
df_station['sub_region'][guarda_mor_index] = None

In [206]:
df_station.rename(
    inplace=True,
    columns={
        'Codigo Estacao': 'station_code',
        'Latitude': 'latitude',
        'Longitude': 'longitude',
        'Altitude': 'altitude',
        'Situacao': 'operational_condition',
        'Data Inicial': 'initial_date',
        'Data Final': 'final_date',
        'Periodicidade da Medicao': 'measurement_frequency',
    }
)

In [207]:
df_station = df_station.astype({
    'station_code': str,
    'latitude': 'Float64',
    'longitude': 'Float64',
    'altitude': 'Float64',
    'initial_date': 'datetime64[ns]',
    'final_date': 'datetime64[ns]',
})

In [208]:
df_station.drop(columns=['Nome'], inplace=True)

In [212]:
df_station.tail()

,station_code,latitude,longitude,altitude,operational_condition,initial_date,final_date,measurement_frequency,city,sub_region
0,A944,-14.828889,-56.441944,195.0,Operante,2019-05-29,2019-12-31,Mensal,ROSARIO OESTE,None
0,B803,-24.570833,-52.800278,598.0,Operante,2016-08-03,2019-12-31,Mensal,CAMPINA DA LAGOA,None
0,B804,-25.371389,-52.400833,889.0,Operante,2016-08-06,2019-12-31,Mensal,LARANJEIRAS DO SUL,None
0,B806,-25.322464,-49.157733,950.0,Operante,2016-05-31,2019-12-31,Mensal,COLOMBO,None
0,F501,-19.98,-43.958611,1199.55,Operante,2013-12-26,2019-12-31,Mensal,BELO HORIZONTE,CERCADINHO


In [213]:
df_station.to_parquet(
    path='data/trusted/environmental/station.parquet.snappy',
    engine='pyarrow',
    index=True,
    compression='snappy',
)

In [214]:
df_meteorology.rename(
    inplace=True,
    columns={
        'Data Medicao': 'measurement_date',
        'NUMERO DE DIAS COM PRECIP. PLUV, MENSAL (AUT)(número)': 'number_of_rainy_days',
        'PRECIPITACAO TOTAL, MENSAL (AUT)(mm)': 'total_precipitation',
        'PRESSAO ATMOSFERICA, MEDIA MENSAL (AUT)(mB)': 'atmospheric_pressure',
        'TEMPERATURA MEDIA, MENSAL (AUT)(°C)': 'average_temperature',
        'VENTO, VELOCIDADE MAXIMA MENSAL (AUT)(m/s)': 'maximum_wind_speed',
        'VENTO, VELOCIDADE MEDIA MENSAL (AUT)(m/s)': 'average_wind_speed',
    }
)

In [215]:
df_meteorology.tail()

,measurement_date,number_of_rainy_days,total_precipitation,atmospheric_pressure,average_temperature,maximum_wind_speed,average_wind_speed,station_code
68,2019-08-31,3,9.4,886.6,17.9,17.1,6.4,F501
69,2019-09-30,2,29.4,883.8,21.0,14.7,6.5,F501
70,2019-10-31,7,69.4,881.9,21.5,14.9,6.0,F501
71,2019-11-30,18,254.4,880.4,21.0,15.0,5.9,F501
72,2019-12-31,19,215.0,881.1,20.9,13.7,4.9,F501


In [216]:
df_meteorology.to_parquet(
    path='data/trusted/environmental/meteorology.parquet.snappy',
    engine='pyarrow',
    index=False,
    compression='snappy',
)